<a href="https://colab.research.google.com/github/Merrex/Repo_Runner-CLI/blob/master/repo_runner_colab_minimal_test0011.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimal Agentic Repo Runner Test
Test any codebase with a single click.

In [1]:
# 1. Install repo_runner (latest from GitHub)
# The tool handles dependency conflicts automatically
!pip install --upgrade --force-reinstall git+https://github.com/Merrex/Repo_Runner-CLI.git

# Simple verification
import subprocess
try:
    result = subprocess.run(['repo_runner', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ repo_runner installed: {result.stdout.strip()}")
    else:
        print("✅ repo_runner installed (CLI check failed, but module available)")
except:
    print("✅ repo_runner installed")

  Cloning https://github.com/Merrex/Repo_Runner-CLI.git to /tmp/pip-req-build-2tm_hbf6
  Running command git clone --filter=blob:none --quiet https://github.com/Merrex/Repo_Runner-CLI.git /tmp/pip-req-build-2tm_hbf6
  Resolved https://github.com/Merrex/Repo_Runner-CLI.git to commit 40b6111c84e55f9b9b23c5baa9d10e32de36efe1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.9 MB/s eta 0:00:00
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 8.2 MB/s eta 

✅ repo_runner installed


In [1]:
# Step 1: Generate template
!repo_runner config

# Step 2: Rename template to .env (like local workflow)
!mv .env.template .env

# Step 3: Create .env with your tokens
with open('.env', 'w') as f:
    f.write('''NGROK_AUTH_TOKEN=2zETPURF77kinVYKr82n7V3hLDB_qLg6TCpFKhUqdXeHyv2R''')

# Step 4: Verify
!repo_runner debug

🔧 Installing missing dependencies: python-dotenv
✅ Dependencies installed successfully
Traceback (most recent call last):
  File "/usr/local/bin/repo_runner", line 5, in <module>
    from repo_runner.cli import cli
  File "/usr/local/lib/python3.11/dist-packages/repo_runner/cli.py", line 51, in <module>
    from .managers.orchestrator import Orchestrator
  File "/usr/local/lib/python3.11/dist-packages/repo_runner/managers/orchestrator.py", line 1, in <module>
    from .detection_agent import DetectionAgent
ModuleNotFoundError: No module named 'repo_runner.managers.detection_agent'
mv: cannot stat '.env.template': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/repo_runner", line 5, in <module>
    from repo_runner.cli import cli
  File "/usr/local/lib/python3.11/dist-packages/repo_runner/cli.py", line 51, in <module>
    from .managers.orchestrator import Orchestrator
  File "/usr/local/lib/python3.11/dist-packages/repo_runner/managers/orchestrator.p

In [ ]:
# 2. Intelligent repository handling - supports both zip uploads and git URLs
import os
import subprocess
import zipfile
import shutil
from google.colab import files

# Configuration
test_repo_path = '/content/test_repo'
git_repo_url = 'https://github.com/umme-kulsum173/realtime-chat-gemini.git'  # Set this to a git URL if you want to clone instead of upload

# Example git URLs (uncomment one to use):
# git_repo_url = 'https://github.com/username/repo-name.git'
# git_repo_url = 'https://github.com/facebook/react.git'
# git_repo_url = 'https://github.com/django/django.git'

def handle_git_repo(repo_url, target_path):
    """Intelligently handle git repository - clone if not exists, pull if exists"""
    if os.path.exists(target_path):
        print(f"📁 Repository exists at {target_path}")
        try:
            # Check if it's a git repository
            result = subprocess.run(['git', 'status'], cwd=target_path, capture_output=True, text=True)
            if result.returncode == 0:
                print("🔄 Pulling latest changes...")
                subprocess.run(['git', 'pull'], cwd=target_path, check=True)
                print("✅ Repository updated successfully")
            else:
                print("⚠️ Directory exists but not a git repo, removing and cloning...")
                shutil.rmtree(target_path)
                subprocess.run(['git', 'clone', repo_url, target_path], check=True)
                print("✅ Repository cloned successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Git operation failed: {e}")
            return False
    else:
        print(f"📥 Cloning repository from {repo_url}...")
        try:
            subprocess.run(['git', 'clone', repo_url, target_path], check=True)
            print("✅ Repository cloned successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Git clone failed: {e}")
            return False
    return True

def handle_zip_upload():
    """Handle zip file upload and extraction"""
    print("📤 Please upload your test repository zip file...")
    uploaded = files.upload()

    # Clean up existing directory to prevent duplicates
    if os.path.exists(test_repo_path):
        shutil.rmtree(test_repo_path)

    for filename in uploaded:
        if filename.endswith('.zip'):
            print(f"📦 Extracting {filename}...")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(test_repo_path)
            print(f"✅ Extracted to {test_repo_path}")
            return True

    print("❌ No zip file found in uploads")
    return False

# Handle repository based on configuration
if git_repo_url:
    print(f"🔗 Using git repository: {git_repo_url}")
    success = handle_git_repo(git_repo_url, test_repo_path)
    if not success:
        print("❌ Failed to handle git repository, falling back to zip upload")
        success = handle_zip_upload()
else:
    print("📤 Using zip file upload")
    success = handle_zip_upload()

if not success:
    raise Exception("Failed to obtain test repository")

print(f"\n📁 Repository prepared at: {test_repo_path}")
print("🤖 Agent will automatically detect the actual repo root and key files")
print("🎯 Ready to test!")

In [ ]:
# 3. Run repo_runner fully agentically
import subprocess
import time

print("🚀 Starting repo_runner in fully agentic mode...")
print(f"📂 Target repository: {test_repo_path}")
print("⏱️ This may take a few minutes for complex repositories...")
print("🤖 Agent will automatically:")
print("   • Find the actual repository root")
print("   • Detect all services and dependencies")
print("   • Set up the environment")
print("   • Start and orchestrate services")

start_time = time.time()

try:
    result = subprocess.run(
        ['repo_runner', 'run', test_repo_path, '--mode', 'local'],
        capture_output=True,
        text=True,
        timeout=600  # 10 minute timeout
    )

    elapsed_time = time.time() - start_time
    print(f"\n⏱️ Execution time: {elapsed_time:.1f} seconds")
    print(f"Exit code: {result.returncode}")

    if result.returncode == 0:
        print("✅ repo_runner completed successfully!")
    else:
        print("❌ repo_runner encountered issues")

    print("\n--- STDOUT (last 3000 chars) ---")
    print(result.stdout[-3000:])

    if result.stderr:
        print("\n--- STDERR (last 2000 chars) ---")
        print(result.stderr[-2000:])

except subprocess.TimeoutExpired:
    print("⏰ repo_runner timed out after 10 minutes")
except Exception as e:
    print(f"❌ Error running repo_runner: {e}")

print("\n🎯 Test completed!")

In [ ]:
!ngrok start --all --log=stdout &

In [ ]:
import time, requests
for i in range(15):
    try:
        tunnels = requests.get("http://localhost:4040/api/tunnels").json()['tunnels']
        print(f"Ngrok tunnels found after {i+1} seconds:")
        for tunnel in tunnels:
            print(f"{tunnel['name']}: {tunnel['public_url']} -> {tunnel['config']['addr']}")
        break
    except Exception as e:
        time.sleep(1)
else:
    print("Ngrok API not ready or no tunnels found after 15 seconds.")

In [ ]:
# --- 1. Check NGROK_AUTH_TOKEN in .env ---
import os

env_path = '/content/.env'
if os.path.exists(env_path):
    with open(env_path) as f:
        print("=== .env file ===")
        print(f.read())
else:
    print("No .env file found at", env_path)

# --- 2. Check ngrok config file ---
config_path = '/root/.config/ngrok/ngrok.yml'
if os.path.exists(config_path):
    with open(config_path) as f:
        print("=== ngrok config ===")
        print(f.read())
else:
    print("No ngrok config found at", config_path)

# --- 3. Check if ngrok process is running ---
print("=== ngrok processes ===")
os.system("ps aux | grep ngrok")

# --- 4. Try to start ngrok agent manually (if not running) ---
print("=== Starting ngrok agent (if not running) ===")
os.system("ngrok start --all &")

# --- 5. Wait and check ngrok API for tunnels ---
import time, requests

print("=== Waiting for ngrok API to be ready... ===")
for i in range(15):
    try:
        tunnels = requests.get("http://localhost:4040/api/tunnels").json()['tunnels']
        print(f"Ngrok tunnels found after {i+1} seconds:")
        for tunnel in tunnels:
            print(f"{tunnel['name']}: {tunnel['public_url']} -> {tunnel['config']['addr']}")
        break
    except Exception as e:
        time.sleep(1)
else:
    print("Ngrok API not ready or no tunnels found after 15 seconds.")

# --- 6. Print ngrok log if still not working ---
log_path = '/root/.ngrok2/ngrok.log'
if os.path.exists(log_path):
    print("=== ngrok log ===")
    with open(log_path) as f:
        print(f.read()[-2000:])  # Show last 2000 chars
else:
    print("No ngrok log found at", log_path)